# Easy Ensemble AdaBoost Classifier

In AdaBoost, a model is trained and then evaluated. After evaluating the errors of the first model, another model is trained. 

The model gives extra weight to the errors from the previous model. The purpose of this weighting is to minimize similar errors in subsequent models. Then, the errors from the second model are given extra weight for the third model. This process is repeated until the error rate is minimized.

However, AdaBoost is more sensetive to overfitting than Random Forests.

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
# from config import db_password
from sqlalchemy import inspect
import matplotlib.pyplot as plt
import pandas as pd

business_df = pd.read_csv("../../../Data/business_census_merged_dataset.csv")


# Categorizing restaurants based on stars ratings
business_df["Category"] = pd.cut(business_df["Stars_Rating"],bins=[0.9,3.5,5],
                                 labels=["Lower Performance", "Higher Performance"])


def changeStatus(status):
    if status == "Lower Performance":
        return 0
    else:
        return 1
    
business_df['Category_Encoded'] = business_df["Category"].apply(changeStatus)
business_df["Category_Encoded"] = pd.to_numeric(business_df["Category_Encoded"])

# Transform Previous State column
def changeStatus(status):
    if status == 'AZ':
        return 1
    elif status == 'CA':
        return 2
    elif status == 'DE':
        return 3
    elif status == 'FL':
        return 4
    elif status == 'ID':
        return 5
    elif status == 'IL':
        return 6
    elif status == 'IN':
        return 7
    elif status == 'LA':
        return 8
    elif status == 'NJ':
        return 9
    elif status == 'NV':
        return 10
    elif status == 'PA':
        return 11
    else:
        return 12
# Along with replace() and map(), encode the state column into numbers.
business_df["State"] = business_df["State"].apply(changeStatus)

business_df["Median_Income(dollars)"] = pd.to_numeric(business_df["Median_Income(dollars)"], errors='coerce')
business_df["Mean_Income(dollars)"] = pd.to_numeric(business_df["Mean_Income(dollars)"], errors='coerce')

business_df = business_df.dropna(subset=['Median_Income(dollars)'])

In [47]:
business_df.columns

Index(['Restaurant_ID', 'Restaurants_Name', 'Address', 'City', 'State',
       'Postal_Code', 'Latitude', 'Longitude', 'Stars_Rating', 'Review_Count',
       'Restaurants_Delivery', 'Outdoor_Seating', 'Accepts_CreditCards',
       'Price_Range', 'Alcohol', 'Good_For_Kids', 'Reservations',
       'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions', 'Total_Estimate_Households_per_Zip',
       'Total_Estimate_Married-couple_Family_households',
       'Total_Estimate_Nonfamily_households', 'Median_Income(dollars)',
       'Mean_Income(dollars)', 'Category', 'Category_Encoded'],
      dtype='object')

In [2]:
# Define the features set
X = business_df[['State', 'Review_Count', 'Restaurants_Delivery', 'Outdoor_Seating',
       'Accepts_CreditCards', 'Price_Range', 'Alcohol', 'Good_For_Kids',
       'Reservations', 'Restaurants_TakeOut', 'WiFi', 'Good_For_Groups',
       'Wheelchair_Accessible', 'Happy_Hour', 'Noise_Level',
       'Dietary_Restrictions',
                'Median_Income(dollars)', 'Total_Estimate_Households_per_Zip']]

In [3]:
# Define the target set
y = business_df["Category_Encoded"]

In [4]:
# Split the model into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y, 
                                                   random_state=1, 
                                                    stratify=y)

In [5]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

model = EasyEnsembleClassifier(n_estimators=100, random_state=1)

model.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [6]:
# Predicting Testing Data
from sklearn.metrics import balanced_accuracy_score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.6671311488624523

In [53]:
# Predicting Training Data
y_pred_train = model.predict(X_train)
balanced_accuracy_score(y_train, y_pred_train)

0.6714445536469451

In [54]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
cm

array([[2041,  945],
       [1108, 2091]])

In [55]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))



                   pre       rec       spe        f1       geo       iba       sup

          0       0.65      0.68      0.65      0.67      0.67      0.45      2986
          1       0.69      0.65      0.68      0.67      0.67      0.45      3199

avg / total       0.67      0.67      0.67      0.67      0.67      0.45      6185

